This is the analysis notebook to reproduce the data in section 2.2. of the thesis.

It answers the following questions:

* pages and revisions:
    * how did the number of pages in enwiktionary evolve over time?
    * when were pages last edited (are there 'zombie' pages)?
    * revisions per page
* contributors:
    * what is the contribution of bots?
    * how big is the contributor community?
* languages:
    * how many languages per entry in most recent slice and over time 
    * how many entries with at least one etymology section in most recent slice and over time
* templates:
    * which are the most common templates in etymology sections over time

In [ ]:
import logging

logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("HistoricalAnalysis")
from pathlib import Path
from collections import Counter
import math
import multiprocessing as mp
from typing import List, Callable
from functools import reduce

from tqdm import tqdm

tqdm.pandas()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker, colors

import etymmap.specific_en

etymmap.specific_en.configure()

from etymmap.wiktionary import DumpProcessor, raw2json
from etymmap.analyze import inspect_json_paths, StatisticsWriter, StatisticsReader
from etymmap.specific import Specific

from utils import cached

In [ ]:
DUMP = DumpProcessor("../dumps/enwiktionary-20220601-pages-meta-history.xml.bz2")
DATA_PATH = Path("./data/enwiktionary-20220601-pages-meta-history")
reader = StatisticsReader(DATA_PATH)

MIN_DATE = "2001-04-01"
MAX_DATE = "2022-07-01"

monthly_timebins = pd.date_range(MIN_DATE, MAX_DATE, freq="MS")
quarterly_timebins = pd.date_range(MIN_DATE, MAX_DATE, freq="QS")
annual_timebins = pd.date_range("2001", "2023", freq="YS")

In [ ]:
# preparation


def get_bot_ids(reader: StatisticsReader):
    contributors = reader.contributors()
    bot_tables = pd.read_html("https://en.wiktionary.org/wiki/Wiktionary:Bots")
    bot_names = pd.concat(
        [table["Bot name"].str.replace(r"\(.*\)", "") for table in bot_tables[1:3]],
        ignore_index=True,
    ).str.strip()
    return (
        contributors[contributors.username.isin(bot_names)].id.astype(int).astype(str)
    )


def prepare_revisions(reader: StatisticsReader):
    """
    Sort the revisions by page_id and timestamp and set page_id as index
    """
    revisions = pd.concat(
        list(
            tqdm(
                reader.revisions(
                    iterator=True,
                    chunksize=10**6,
                    # put sort columns to the left
                    usecols=["id", "contributor", "minor", "page_id", "timestamp"],
                ),
                total=66,
                unit=" chunks",
            )
        )
    )
    revisions.sort_values(["page_id", "timestamp"], inplace=True)
    revisions.set_index("page_id", inplace=True)
    return revisions


# utility


def number_format(x, pos):
    for k, (s, r) in {10**6: ("M", 1), 1000: ("k", 0), 0: ("", 0)}.items():
        if x >= k:
            if r:
                return f"{round(x / k, r)}{s}"
            return f"{int(x) // (k or 1)}{s}"
    return x


numberFormatter = ticker.FuncFormatter(number_format)

bot_ids = cached(lambda: get_bot_ids(reader), DATA_PATH / "bot_ids.pickle")


def read_main_ns_page_ids():
    pages = reader.pages(usecols=["id", "namespace"])
    return pages[pages.namespace.isin([0, 118])].id


def last_idx(size):
    ret = np.zeros(size, dtype=bool)
    ret[-1] = True
    return ret


def get_revisions(
    only_major=True,
    keep_last=True,
    only_main=True,
    only_human=False,
    only_last_per_timebin=False,
    timebins=monthly_timebins,
    head=None,
    columns=None,
):
    revisions = pd.read_pickle(DATA_PATH / "revisions.pickle").head(head)

    if only_main:
        logger.info(f"Filter main namespace revisions ({len(revisions)})")
        main_ns_page_ids = read_main_ns_page_ids()
        revisions = revisions[revisions.index.isin(main_ns_page_ids)]
        logger.info(f"Keep {len(revisions)}")

    if only_human:
        logger.info(f"Filter bot revisions ({len(revisions)})")
        revisions = revisions[~revisions.contributor.isin(bot_ids)]
        logger.info(f"Keep {len(revisions)}")

    if only_major:
        logger.info(f"Filter minor revisions ({len(revisions)})")
        if keep_last:
            last = revisions.groupby("page_id").tail(1).id
            revisions = revisions[~revisions.minor | revisions.id.isin(last)]
        else:
            revisions = revisions[~revisions.minor]
        logger.info(f"Keep {len(revisions)}")

    revisions["timebin"] = pd.cut(
        revisions.timestamp, bins=timebins, labels=timebins[:-1]
    )
    if only_last_per_timebin:
        logger.info(f"Filter last per timebin ({len(revisions)})")
        # we can use the tail as timestamps are sorted
        revisions = revisions.groupby(["page_id", "timebin"], observed=True).tail(1)
        logger.info(f"Keep {len(revisions)}")
    return revisions[columns] if columns else revisions

# Compile the statistics

In [ ]:
# takes ~4 hours with 8 cores intel i7

# writer = StatisticsWriter()
# writer.write_stats(DUMP,
#                   out="./data/enwiktionary-20220601-pages-meta-history",
#                   chunksize_bytes=500*10**6,
#                   progress={"unit": "pages", "total": 8.16*10**6},
#                   mp_processes=6)


# ! requires lot of memory for sorting (~ 10 GB)
_ = cached(prepare_revisions, DATA_PATH / "revisions.pickle")

## Page statistics

In [ ]:
pages = reader.pages()

In [ ]:
# total number of pages?
len(pages)

In [ ]:
# number of pages per namespace?
hist = pages.namespace.value_counts()
pd.DataFrame({"count": hist, "%": round(100 * hist / len(pages), 3)})

In [ ]:
# creation, last edited, number of revisions per page
page_stats = (
    get_revisions(only_main=True, only_human=False, only_major=True, keep_last=True)
    .timestamp.groupby(level="page_id", sort=False)
    .agg(["min", "max", "count"])
    .rename(columns={"min": "created", "max": "last_edited", "count": "n_revisions"})
)

page_stats["created_bins"] = pd.cut(
    page_stats.created, bins=monthly_timebins, labels=monthly_timebins[:-1]
)
page_stats["last_edited_bins"] = pd.cut(
    page_stats.last_edited, bins=monthly_timebins, labels=monthly_timebins[:-1]
)

In [ ]:
# number of pages over time and last edit histogram
created = page_stats.created_bins.value_counts().sort_index()
last_edited = page_stats.last_edited_bins.value_counts().sort_index()
created.index = pd.to_datetime(created.index).date
last_edited.index = pd.to_datetime(last_edited.index).date
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
xticks = pd.date_range("2001", "2022", freq=pd.DateOffset(years=3))
created.cumsum().plot(ax=axes[0], grid=True, xticks=xticks)
last_edited.plot(drawstyle="steps", ax=axes[1], grid=True, xticks=xticks)
for a in axes:
    a.set_xticklabels(xticks.year)
    a.yaxis.set_major_formatter(numberFormatter)
plt.savefig("pages.pdf")

In [ ]:
# number of major edits per month
increase_per_month = page_stats.last_edited_bins.value_counts()
increase_per_month

In [ ]:
def most_active_users(months):
    ret = []
    contributors = reader.contributors()
    contributors = contributors[contributors.id.notnull()]
    contributors["id"] = contributors.id.astype(int).astype(str)
    revisions = get_revisions(
        only_main=True, only_human=False, only_major=True, keep_last=True
    )
    for month in months:
        revisions_of_month = revisions[revisions.timebin == month]
        edits_per_contributor = revisions_of_month.contributor.value_counts()
        user = edits_per_contributor.index[0]
        username = contributors[contributors.id == user].username.iloc[0]
        number_of_edits = edits_per_contributor[0]
        ratio = round(100 * number_of_edits / len(revisions_of_month), 3)
        ret.append((month, user, username, number_of_edits, ratio))
    return pd.DataFrame.from_records(
        ret, columns=["month", "id", "username", "n_edits", "% month"]
    )

In [ ]:
most_active_users(increase_per_month.index[:10])

In [ ]:
all_revisions = get_revisions(
    only_main=True, only_human=False, only_major=False, columns=["timebin"]
)

In [ ]:
non_bot_revisions = get_revisions(
    only_main=True, only_human=True, only_major=False, columns=["timebin"]
)

In [ ]:
# how much of the pages are edited by bots?
round(1 - (len(non_bot_revisions) / len(all_revisions)), 3)

In [ ]:
# how many pages are only edited by bots?
round(1 - non_bot_revisions.index.unique().size / all_revisions.index.unique().size, 3)

In [ ]:
page_stats_nonbots = non_bot_revisions.timebin.groupby("page_id").agg(
    ["min", "max", "count"]
)

In [ ]:
# last edits for nonbots
last_edited = (
    page_stats_nonbots["max"].value_counts().rename("last_edited").sort_index()
)
last_edited.index = pd.to_datetime(last_edited.index).date
last_edited.plot(drawstyle="steps", figsize=(14, 6), grid=True, xticks=xticks)
_ = plt.gca().set_xticklabels(xticks.year)

In [ ]:
# same, but cumsum
(last_edited.cumsum() / last_edited.sum()).plot(grid=True)

In [ ]:
# x-axis: number of edits, y-axis: number of users that made this many edits
data = page_stats_nonbots["count"].hist(bins=range(25))

In [ ]:
# most (human) edited pages
(
    page_stats_nonbots[["count"]]
    .sort_values("count", ascending=False)
    .merge(
        reader.pages(usecols=["id", "title"]),
        how="left",
        left_on="page_id",
        right_on="id",
    )
).head(10)

# Contributors

In [ ]:
# how many total contributors?
len(
    get_revisions(
        only_main=False, only_human=False, only_major=False, columns=["contributor"]
    )
)

In [ ]:
# revisions per contributor
revisions = get_revisions(
    only_main=True,
    only_human=False,
    only_major=False,
    columns=["timebin", "contributor", "minor"],
)

In [ ]:
def get_user_stats(d):
    return pd.Series(
        {
            "contributions": len(d),
            "edited_pages": d.index.unique().size,
            "major_contributions": len(d[~d.minor]),
        }
    )

In [ ]:
# contributor_stats = revisions.groupby("contributor").progress_apply(get_user_stats)
# contributor_stats.to_pickle(DATA_PATH / "contributor_stats.pickle")
contributor_stats = pd.read_pickle(DATA_PATH / "contributor_stats.pickle")

In [ ]:
contributor_stats

In [ ]:
total = contributor_stats.contributions.sum()
total

In [ ]:
contributor_stats["is_bot"] = pd.Series(
    contributor_stats.index.isin(bot_ids), index=contributor_stats.index
).replace({True: 1, False: -1})

In [ ]:
# most contributors have very few edits
contributor_stats.contributions.hist(bins=range(25))

In [ ]:
contributors = reader.contributors()
contributors.id = contributors.id.fillna(-1).astype(int).astype(str)
contributor_stats = contributor_stats.merge(
    contributors[["id", "username"]], left_index=True, right_on="id", how="left"
)
contributor_stats.set_index("id", inplace=True)

In [ ]:
contributor_stats[contributor_stats.is_bot == -1].sort_values(
    "contributions", ascending=False
)

In [ ]:
maybe_bot = (contributor_stats.is_bot == -1) & contributor_stats.username.str.match(
    ".*[Bb]ot$"
).fillna(False)
contributor_stats.is_bot.loc[maybe_bot] = 0

contributor_stats.is_bot.loc[contributor_stats.username.isna()] = -2

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
by_group = contributor_stats.groupby("is_bot").contributions.sum() / total
by_group.index = ["unregistered", "human", "maybe bot", "bot"]
by_group.plot(kind="bar", ax=axs[0])
axs[0].set_xticklabels(by_group.index.values, rotation=0)
axs[0].grid(axis="y")

topk = [10, 100, 1000, 100000]
contributions = contributor_stats[
    contributor_stats.is_bot < 0
].contributions.sort_values(ascending=False)
topk_ratios = pd.Series(
    {i: contributions.iloc[:i].sum() / contributions.sum() for i in topk}
)
topk_ratios.plot(ax=axs[1], kind="bar")
axs[1].grid(axis="y")
_ = axs[1].set_xticklabels(topk, rotation=0)
plt.savefig("users.pdf")

# Languages and Templates

In [ ]:
NWORKERS = 8
SPLIT = 512
BINS = annual_timebins


def split_entries(entries, n=8):
    groups = entries.groupby("page_id").ngroup()
    group_chunks = groups // math.ceil(groups.iloc[-1] / n)
    starts = [(group_chunks == i).argmax() for i in range(n)]
    for start, end in zip(starts, starts[1:] + [len(entries)]):
        yield entries.iloc[start:end]


def prepare_entries(
    reader, columns=["revision_id", "language", "has_etymology"], split=True
):
    logger.info("Read and filter revisions")
    revisions = get_revisions(
        only_last_per_timebin=True, columns=["id", "timebin"], timebins=BINS
    )

    logger.info("Read entries")
    langs = reader.entries(usecols=columns, index_col="revision_id")

    logger.info("Merging...")
    entries = revisions.merge(langs, left_on="id", right_index=True, how="left")
    logger.info("Timebins as datetime")
    entries.timebin = pd.to_datetime(entries.timebin)

    if split:
        logger.info(f"Split into {SPLIT} frames")
        return list(split_entries(entries, n=SPLIT))
    return entries

## Languages

In [ ]:
entries = prepare_entries(reader)

In [ ]:
sum([e.memory_usage() for e in entries]).sum() / 10**9

In [ ]:
def languages_per_page(page_df):
    return (
        page_df[["timebin", "language"]]
        .groupby("timebin")
        .language
        # collect all languages in an array per timebin
        .apply(lambda s: (None if s.empty else s.values))
        # we need a forward fill for all timebins
        .reindex(BINS)
        .fillna(method="ffill")
        .explode()
    )


def etym_languages_per_page(page_df):
    return (
        page_df[["timebin", "language", "has_etymology"]]
        .groupby("timebin")[["language", "has_etymology"]]
        # collect all languages in an array per timebin
        # don't ask me why pandas cannot handle the array as above
        .apply(
            lambda df: (None if df.empty else df[df.has_etymology].language.tolist())
        )
        # we need a forward fill for all timebins
        .reindex(BINS)
        .fillna(method="ffill")
        .explode()
    )


def languages_count(_entries, with_etym=False):
    return (
        _entries.groupby("page_id")
        .apply(etym_languages_per_page if with_etym else languages_per_page)
        .groupby(level="timebin")
        .value_counts()
        .unstack(level="language")
    )


def etym_languages_count(_entries):
    return languages_count(_entries, True)


def chunkwise_languages_count(
    entries: List[pd.DataFrame], count_chunk: Callable = languages_count
):
    with mp.Pool(NWORKERS) as pool:
        counts = list(tqdm(pool.imap_unordered(count_chunk, entries), total=SPLIT))
    all_counts = pd.DataFrame(
        0, index=BINS, columns=set(c for d in counts for c in d.columns)
    )
    for count in counts:
        all_counts = all_counts.add(count, fill_value=0)
    languages = all_counts.iloc[-1].sort_values(ascending=False)
    return all_counts[languages.index]

In [ ]:
SPLIT = 512
BINS = monthly_timebins

# language_counts = chunkwise_languages_count(entries[:8])
# language_counts.to_pickle(DATA_PATH / "language_counts.pickle")
language_counts = pd.read_pickle(DATA_PATH / "language_counts.pickle")

In [ ]:
# etym_language_counts = chunkwise_language_count(entries, etym_languages_count)
# etym_language_counts.to_pickle(DATA_PATH / "etym_language_counts.pickle")
etym_language_counts = pd.read_pickle(DATA_PATH / "etym_language_counts.pickle")

## Current most common languages

In [ ]:
lm = Specific.language_mapper

current_lcs = language_counts.iloc[-1]
current_elcs = etym_language_counts.iloc[-1]

In [ ]:
current_lcs.sum(), current_lcs[
    ~pd.Series(current_lcs.index, index=current_lcs.index).str.startswith("?")
].sum()

In [ ]:
top30 = current_lcs.astype(int).iloc[[*range(1, 31), 100, 500]].copy()
top30.index = [lm.code2name(l) for l in top30.index]
top30.reset_index()

In [ ]:
current_lcs[current_lcs <= 10]

In [ ]:
rare = current_lcs[current_lcs <= 10].index[0]
lm.code2name(rare), (current_lcs.index == rare).argmax()

In [ ]:
(~pd.Series(current_lcs.index).str.startswith("?")).sum()

In [ ]:
[len(current_lcs[current_lcs >= i]) for i in [1000, 100, 10]]

In [ ]:
current_lcs[
    ~pd.Series(current_lcs.index, index=current_lcs.index).str.startswith("?")
].sum()

In [ ]:
current_elcs.sum(), current_elcs[
    ~pd.Series(current_elcs.index, index=current_elcs.index).str.startswith("?")
].sum()

In [ ]:
top30 = current_elcs.astype(int).iloc[[*range(50), 500]].copy()
top30.index = [lm.code2name(l) for l in top30.index]
top30.reset_index()

In [ ]:
rare = current_elcs[current_elcs <= 10].index[0]
lm.code2name(rare), (current_elcs.index == rare).argmax()

In [ ]:
[len(current_elcs[current_elcs >= i]) for i in [1000, 100, 10]]

# Languages over time

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

default_colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
langcolor = {
    l: default_colors[i % len(default_colors)]
    for i, l in enumerate(language_counts.columns[1:])
}

language_counts.iloc[:, 1:11].plot(
    ax=axs[0],
    color=[langcolor.get(c, default_colors[-1]) for c in language_counts.columns[1:11]],
)
axs[0].grid(axis="y")
axs[0].yaxis.set_major_formatter(numberFormatter)

etym_language_counts.iloc[:, :10].plot(
    ax=axs[1],
    color=[
        langcolor.get(c, default_colors[-1]) for c in etym_language_counts.columns[:10]
    ],
)
axs[1].grid(axis="y")
axs[1].yaxis.set_major_formatter(numberFormatter)
plt.savefig("languages.pdf")

## Templates

In [ ]:
SPLIT = 1024
BINS = quarterly_timebins

entries = prepare_entries(
    reader, columns=["revision_id", "language", "templates", "has_etymology"]
)

In [ ]:
def combine(per_timebin):
    def combine_(s):
        return reduce(list.__iadd__, s)

    if per_timebin.empty:
        return None
    else:
        per_timebin = per_timebin[
            per_timebin.templates.notnull() & per_timebin.has_etymology
        ]
        if per_timebin.empty:
            # the entry at this timebin does not have etymology templates
            return []
        elif len(per_timebin) == 1:
            return per_timebin.templates.iloc[0]
        else:
            # only sometimes, there are multiple entries of the same language
            # then build one list containing all elements
            return per_timebin.templates.agg(combine_)


def combine_templates(df):
    return df.groupby(["page_id", "language", "timebin"], observed=True)[
        ["templates", "has_etymology"]
    ].apply(combine)


def ffill(df):
    return df.groupby(["page_id", "language"]).apply(
        lambda s: pd.Series(
            s.droplevel(["page_id", "language"]), index=named_index
        ).fillna(method="ffill")
    )


named_index = pd.Series(BINS, name="timebin")


def etymology_template_counts(chunk: pd.DataFrame):
    chunk["templates"] = chunk.templates.replace("", None).str.split("|")
    templates_per_timebin = (ffill(combine_templates(chunk))).droplevel(
        ["page_id", "language"]
    )

    s = templates_per_timebin.dropna()
    s = s[s.apply(len) > 1].explode().str.split("=")
    has_two = s.apply(len) == 2
    bad_entries = s[~has_two]
    if not bad_entries.empty:
        logger.warning(bad_entries.tolist())
    s = s[has_two]
    df = pd.DataFrame.from_records(s, index=s.index, columns=["template", "count"])
    df["count"] = df["count"].astype(int)
    return df.pivot_table(
        index="timebin", columns="template", values="count", aggfunc="sum", fill_value=0
    )


def chunkwise_template_counts(entries: List[pd.DataFrame]):
    with mp.Pool(NWORKERS) as pool:
        counts = list(
            tqdm(pool.imap_unordered(etymology_template_counts, entries), total=SPLIT)
        )
    all_counts = pd.DataFrame(
        0, index=BINS, columns=set(c for d in counts for c in d.columns)
    )
    for count in counts:
        all_counts = all_counts.add(count, fill_value=0)
    templates = all_counts.iloc[-1].sort_values(ascending=False)
    return all_counts[templates.index]

In [ ]:
# template_counts = chunkwise_template_counts(entries)
# template_counts.to_pickle(DATA_PATH / "template_counts.pickle")
template_counts = pd.read_pickle(DATA_PATH / "template_counts.pickle")

In [ ]:
template_counts.iloc[-1].sort_values(ascending=False)[:32]

In [ ]:
ts = template_counts.sum().sort_values(ascending=False).index

In [ ]:
groups = [template_counts[ts].iloc[:, i * 8 : (i + 1) * 8] for i in range(4)]

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

for group, ax in zip(groups, axs.flatten()):
    group.plot(ax=ax)
    ax.yaxis.set_major_formatter(numberFormatter)
plt.savefig("templates.pdf")